In [1]:
import acquire
import numpy as np
import pandas as pd
from env import username, password, host
import os
from sklearn import datasets
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics

import warnings
warnings.filterwarnings("ignore")

from pydataset import data

from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

# Titanic Dataset

### 1. What is your baseline prediction? What is your baseline accuracy? remember: your baseline prediction for a classification problem is predicting the most prevelant class in the training dataset (the mode). When you make those predictions, what is your accuracy? This is your baseline accuracy.

In [206]:
titanic = acquire.grab_titanic_data()
titanic.head()

,Unnamed: 0,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,deck,embark_town,alone
0,0,0,0,3,male,22.0,1,0,7.2500,S,Third,NaN,Southampton,0
1,1,1,1,1,female,38.0,1,0,71.2833,C,First,C,Cherbourg,0
2,2,2,1,3,female,26.0,0,0,7.9250,S,Third,NaN,Southampton,1
3,3,3,1,1,female,35.0,1,0,53.1000,S,First,C,Southampton,0
4,4,4,0,3,male,35.0,0,0,8.0500,S,Third,NaN,Southampton,1


In [207]:
titanic = acquire.prep_titanic(titanic)
titanic.rename(columns={'sex': 'isMale'}, inplace=True)
titanic.head()

,survived,pclass,isMale,age,sibsp,parch,fare,embark_town,alone
0,0,3,male,22.0,1,0,7.2500,1.0,0
1,1,1,female,38.0,1,0,71.2833,2.0,0
2,1,3,female,26.0,0,0,7.9250,1.0,1
3,1,1,female,35.0,1,0,53.1000,1.0,0
4,0,3,male,35.0,0,0,8.0500,1.0,1


In [187]:
print(titanic.age.mean())
print(titanic.age.mode())
print(titanic.age.median())

29.36158249158249
0    28.0
Name: age, dtype: float64
28.0


In [208]:
titanic.age[titanic.age.isnull()] = 28
titanic.head(10)

,survived,pclass,isMale,age,sibsp,parch,fare,embark_town,alone
0,0,3,male,22.0,1,0,7.2500,1.0,0
1,1,1,female,38.0,1,0,71.2833,2.0,0
2,1,3,female,26.0,0,0,7.9250,1.0,1
3,1,1,female,35.0,1,0,53.1000,1.0,0
4,0,3,male,35.0,0,0,8.0500,1.0,1
5,0,3,male,28.0,0,0,8.4583,3.0,1
6,0,1,male,54.0,0,0,51.8625,1.0,1
7,0,3,male,2.0,3,1,21.0750,1.0,0
8,1,3,female,27.0,0,2,11.1333,1.0,0
9,1,2,female,14.0,1,0,30.0708,2.0,0


In [209]:
#titanic[titanic.age == 'NaN'] = 28

In [210]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   survived     891 non-null    int64  
 1   pclass       891 non-null    int64  
 2   isMale       891 non-null    object 
 3   age          891 non-null    float64
 4   sibsp        891 non-null    int64  
 5   parch        891 non-null    int64  
 6   fare         891 non-null    float64
 7   embark_town  889 non-null    float64
 8   alone        891 non-null    int64  
dtypes: float64(3), int64(5), object(1)
memory usage: 62.8+ KB


In [211]:
titanic.embark_town.value_counts()

1.0    644
2.0    168
3.0     77
Name: embark_town, dtype: int64

In [212]:
titanic.embark_town[titanic.embark_town.isnull()] = 1.0

In [213]:
titanic.isMale[titanic.isMale == 'male'] = 1
titanic.isMale[titanic.isMale == 'female'] = 0
titanic.head(10)

,survived,pclass,isMale,age,sibsp,parch,fare,embark_town,alone
0,0,3,1,22.0,1,0,7.2500,1.0,0
1,1,1,0,38.0,1,0,71.2833,2.0,0
2,1,3,0,26.0,0,0,7.9250,1.0,1
3,1,1,0,35.0,1,0,53.1000,1.0,0
4,0,3,1,35.0,0,0,8.0500,1.0,1
5,0,3,1,28.0,0,0,8.4583,3.0,1
6,0,1,1,54.0,0,0,51.8625,1.0,1
7,0,3,1,2.0,3,1,21.0750,1.0,0
8,1,3,0,27.0,0,2,11.1333,1.0,0
9,1,2,0,14.0,1,0,30.0708,2.0,0


In [214]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   survived     891 non-null    int64  
 1   pclass       891 non-null    int64  
 2   isMale       891 non-null    object 
 3   age          891 non-null    float64
 4   sibsp        891 non-null    int64  
 5   parch        891 non-null    int64  
 6   fare         891 non-null    float64
 7   embark_town  891 non-null    float64
 8   alone        891 non-null    int64  
dtypes: float64(3), int64(5), object(1)
memory usage: 62.8+ KB


In [215]:
t_train, t_validate, t_test = acquire.split(titanic)
t_train.shape, t_validate.shape, t_test.shape

((534, 9), (178, 9), (179, 9))

In [216]:
(t_train.survived == 1).mean()

0.40074906367041196

## What is your baseline prediction?  What is your baseline accuracy? 

In [217]:
t_train['baseline'] = 0

In [218]:
t_train.baseline.value_counts()

0    534
Name: baseline, dtype: int64

In [219]:
#t_train = t_train.drop(columns='Unnamed: 0')
t_train.head()

,survived,pclass,isMale,age,sibsp,parch,fare,embark_town,alone,baseline
304,0,3,1,28.0,0,0,8.0500,1.0,1,0
718,0,3,1,28.0,0,0,15.5000,3.0,1,0
102,0,1,1,21.0,0,1,77.2875,1.0,0,0
737,1,1,1,35.0,0,0,512.3292,2.0,1,0
269,1,1,0,35.0,0,0,135.6333,1.0,1,0


In [220]:
print(f'The Baseline accuracy is: {metrics.accuracy_score(t_train.survived, t_train.baseline)}, This is for non-survival')

The Baseline accuracy is: 0.599250936329588, This is for non-survival


### 2. Fit the decision tree classifier to your training sample and transform (i.e. make predictions on the training sample)

In [221]:
#t_train.sex[t_train.sex == 'male'] = 1
#t_train.sex[t_train.sex == 'female'] = 0
#t_train[t_train.sex == 'female'] = 0
#t_train.rename(columns={'sex': 'isMale'})
t_train.head()

,survived,pclass,isMale,age,sibsp,parch,fare,embark_town,alone,baseline
304,0,3,1,28.0,0,0,8.0500,1.0,1,0
718,0,3,1,28.0,0,0,15.5000,3.0,1,0
102,0,1,1,21.0,0,1,77.2875,1.0,0,0
737,1,1,1,35.0,0,0,512.3292,2.0,1,0
269,1,1,0,35.0,0,0,135.6333,1.0,1,0


In [222]:
#t_train = t_train.rename(columns={'sex': 'isMale'})

In [223]:
x_tr = t_train.drop(columns='survived')
y_tr = t_train.survived

x_val = t_validate.drop(columns='survived')
y_val = t_validate.survived

x_test = t_test.drop(columns='survived')
y_test = t_test.survived

x_tr.head()

,pclass,isMale,age,sibsp,parch,fare,embark_town,alone,baseline
304,3,1,28.0,0,0,8.0500,1.0,1,0
718,3,1,28.0,0,0,15.5000,3.0,1,0
102,1,1,21.0,0,1,77.2875,1.0,0,0
737,1,1,35.0,0,0,512.3292,2.0,1,0
269,1,0,35.0,0,0,135.6333,1.0,1,0


In [224]:
x_tr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 534 entries, 304 to 133
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   pclass       534 non-null    int64  
 1   isMale       534 non-null    object 
 2   age          534 non-null    float64
 3   sibsp        534 non-null    int64  
 4   parch        534 non-null    int64  
 5   fare         534 non-null    float64
 6   embark_town  534 non-null    float64
 7   alone        534 non-null    int64  
 8   baseline     534 non-null    int64  
dtypes: float64(3), int64(5), object(1)
memory usage: 41.7+ KB


In [230]:
t_classf = DecisionTreeClassifier()
t_classf = t_classf.fit(x_tr, y_tr)

y_predict = t_classf.predict(x_tr)
y_predict_prob = t_classf.predict_proba(x_tr)

In [231]:
t_classf.fit(x_tr, y_tr)

DecisionTreeClassifier()

### 3. Evaluate your in-sample results using the model score, confusion matrix, and classification report.

In [233]:
print(f'Model score: {t_classf.score(x_tr, y_tr)}')

Model score: 0.9868913857677902


In [234]:
cm = confusion_matrix(y_tr, y_predict)
print(pd.DataFrame(cm))

     0    1
0  319    1
1    6  208


In [237]:
print(classification_report(y_tr, y_predict))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       320
           1       1.00      0.97      0.98       214

    accuracy                           0.99       534
   macro avg       0.99      0.98      0.99       534
weighted avg       0.99      0.99      0.99       534



### 4. Compute: Accuracy, true positive rate, false positive rate, true negative rate, false negative rate, precision, recall, f1-score, and support.

In [ ]:
# accuracy
print('------- Model 1 -----------')
print('Accuracy score is: ', accuracy_score(y_train, y_pred))

In [ ]:
# true positive rate, false positive rate, true negative rate, false negative rate
tp = cm[0,0]
tn = cm[1,1]
fp = cm[0,1]
fn = cm[1,0]
print('------- Model 1 -----------')
print('True positive rate is:', tp/(tp+tn+fp+fn))
print('False positive rate is', fp/(tp+tn+fp+fn))
print('True negative rate is', tn/(tp+tn+fp+fn))
print('False negative rate is', fn/(tp+tn+fp+fn))

In [ ]:
# precision, recall, f1-score and support
print('------- Model 1 -----------')
print('Precision is: ', precision_score(y_train, y_pred))
print('Recall is:', recall_score(y_train, y_pred))
print('f1 score is:', f1_score(y_train, y_pred))
print('Support is 0:', tp+fp)
print('           1:', tn+fn)

### 5. Run through steps 2-4 using a different max_depth value.

### 6. Which model performs better on your in-sample data?

### 7. Which model performs best on your out-of-sample data, the validate set?

### 1. Work through these same exercises using the Telco dataset.
### 2. Experiment with this model on other datasets with a higher number of output classes.

In [ ]:
# changing max depth to 5
clf2 = DecisionTreeClassifier(max_depth=5)

clf2 = clf2.fit(X_train, y_train)

y_pred2 = clf2.predict(X_train)
y_pred_prob2 = clf2.predict_proba(X_train)

In [ ]:
# model score
print('------- Model 2 -----------')
print(f'training score for model2: {clf2.score(X_train, y_train):.2%}')

In [ ]:
# confusion matrix
cm2 = confusion_matrix(y_train, y_pred2)
print(pd.DataFrame(cm2))

In [ ]:
# classification report
print('------- Model 2 -----------')
print(classification_report(y_train, y_pred2))

In [ ]:
# accuracy
print('------- Model 2 -----------')
print('Accuracy score for model 2 is: ', accuracy_score(y_train, y_pred2))

In [ ]:
# true positive rate, false positive rate, true negative rate, false negative rate
tp2 = cm2[0,0]
tn2 = cm2[1,1]
fp2 = cm2[0,1]
fn2 = cm2[1,0]

print('------- Model 2 -----------')
print('True positive rate is:', tp2/(tp2+tn2+fp2+fn2))
print('False positive rate is', fp2/(tp2+tn2+fp2+fn2))
print('True negative rate is', tn2/(tp2+tn2+fp2+fn2))
print('False negative rate is', fn2/(tp2+tn2+fp2+fn2))

In [ ]:
# precision, recall, f1-score and support
print('------- Model 2 -----------')
print('Precision is: ', precision_score(y_train, y_pred2))
print('Recall is:', recall_score(y_train, y_pred2))
print('f1 score is:', f1_score(y_train, y_pred2))
print('Support is 0:', tp2+fp2)
print('           1:', tn2+fn2)

## Which model performs better on your in-sample data?

Model 1 performs better when evaluating performance against the in-sample (training) data

## Which model performs best on your out-of-sample data, the validate set?

In [ ]:
print('------- Model 1 -----------')
print('Accuracy of Decision Tree classifier on validate set: {:.2f}'
     .format(clf.score(X_validate, y_validate)))
print('')
print('------- Model 2 -----------')
print('Accuracy of Decision Tree classifier on validate set: {:.2f}'
     .format(clf2.score(X_validate, y_validate)))


In [ ]:
# classification report
y_pred = clf.predict(X_validate)
y_pred2 = clf2.predict(X_validate)

print('------- Model 1 -----------')
print(classification_report(y_validate, y_pred))

print('------- Model 2 -----------')
print(classification_report(y_validate, y_pred2))

Model 2 performs slightly better with a max depth of 5

# Work through these same exercises using the Telco dataset.
    ## building two models simultaneously

In [ ]:
# get and prepare iris data
iris = acquire.get_iris_data()
iris = prepare.prep_iris(iris)
i_train, i_validate, i_test = prepare.my_split(iris, target='species')
i_train.shape, i_validate.shape, i_test.shape

In [ ]:
# find the baseline and the baseline accuracy
baseline = i_train.species.value_counts().idxmax()
print('The most common value (our baseline) is:', baseline)
print('The rate of occurance (our baseline accuracy) is:', len(i_train[i_train.species == baseline]) / len(i_train) * 100)

In [ ]:
# split each of our datasets into X and y 

X_train = i_train.drop(columns=('species'))
y_train = i_train.species

X_validate = i_validate.drop(columns=('species'))
y_validate = i_validate.species

X_test = i_test.drop(columns='species')
y_test = i_test.species

In [ ]:
# create and fit the models
clf1 = DecisionTreeClassifier(max_depth=8)
clf2 = DecisionTreeClassifier(max_depth=4)

clf1 = clf1.fit(X_train, y_train)
clf2 = clf2.fit(X_train, y_train)

y_pred1 = clf1.predict(X_train)
y_pred2 = clf2.predict(X_train)

y_pred_prob1 = clf1.predict_proba(X_train)
y_pred_prob2 = clf2.predict_proba(X_train)

In [ ]:
# model score
print(f'Model 1 training score: {clf1.score(X_train, y_train):.2%}')
print(f'Model 2 training score: {clf2.score(X_train, y_train):.2%}')

In [ ]:
# confusion matrix
cm1 = confusion_matrix(y_train, y_pred1)
cm2 = confusion_matrix(y_train, y_pred2)
print('----Model 1----')
print(pd.DataFrame(cm1))
print('')
print('----Model 2----')
print(pd.DataFrame(cm2))

In [ ]:
# classification report
print(classification_report(y_train, y_pred1))
print(classification_report(y_train, y_pred2))

## ...and comparing with validation data

In [ ]:
print('------- Model 1 -----------')
print('Accuracy of Decision Tree classifier on validate set: {:.2f}'
     .format(clf1.score(X_validate, y_validate)))
print('')
print('------- Model 2 -----------')
print('Accuracy of Decision Tree classifier on validate set: {:.2f}'
     .format(clf2.score(X_validate, y_validate)))

In [ ]:
# classification report
y_pred1 = clf1.predict(X_validate)
y_pred2 = clf2.predict(X_validate)

print('------- Model 1 -----------')
print(classification_report(y_validate, y_pred1))

print('------- Model 2 -----------')
print(classification_report(y_validate, y_pred2))

The second model performs better with max depth of 4 versus max depth of 8 for the first model

# Experiment with this model on other datasets with a higher number of output classes.


In [ ]:
# The glass dataset contains 9 columns quantifying the contents of 9 different elements in a glass sample.  
# The 10th column is the class of the glass, and integer from 1-7.

glass = pd.read_csv('glass.csv')
glass.head()

In [ ]:
glass.columns =['ri', 'na', 'mg', 'al', 'si', 'k','ca','ba','fe', 'class']

In [ ]:
# split into train, validate, test
g_train, g_validate, g_test = prepare.my_split(glass, target='class')
g_train.shape, g_validate.shape, g_test.shape

In [ ]:
# find the baseline and the baseline accuracy
baseline = g_train['class'].value_counts().idxmax()
print('The most common value (our baseline) is:', baseline)
print('The rate of occurance (our baseline accuracy) is:', len(g_train[g_train['class'] == baseline]) / len(g_train) * 100)

In [ ]:
# split each of our datasets into X and y 

X_train = g_train.drop(columns=('class'))
y_train = g_train['class']

X_validate = g_validate.drop(columns=('class'))
y_validate = g_validate['class']

X_test = g_test.drop(columns='class')
y_test = g_test['class']

In [ ]:
# create and fit the models
clf1 = DecisionTreeClassifier(max_depth=8)
clf2 = DecisionTreeClassifier(max_depth=4)

clf1 = clf1.fit(X_train, y_train)
clf2 = clf2.fit(X_train, y_train)

y_pred1 = clf1.predict(X_train)
y_pred2 = clf2.predict(X_train)

y_pred_prob1 = clf1.predict_proba(X_train)
y_pred_prob2 = clf2.predict_proba(X_train)

In [ ]:
# model score
print(f'Model 1 training score: {clf1.score(X_train, y_train):.2%}')
print(f'Model 2 training score: {clf2.score(X_train, y_train):.2%}')

In [ ]:
# confusion matrix
cm1 = confusion_matrix(y_train, y_pred1)
cm2 = confusion_matrix(y_train, y_pred2)
print('----Model 1----')
print(pd.DataFrame(cm1))
print('')
print('----Model 2----')
print(pd.DataFrame(cm2))

In [ ]:
# classification report
print(classification_report(y_train, y_pred1))
print(classification_report(y_train, y_pred2))

In [ ]:
print('------- Model 1 -----------')
print('Accuracy of Decision Tree classifier on validate set: {:.2f}'
     .format(clf1.score(X_validate, y_validate)))
print('')
print('------- Model 2 -----------')
print('Accuracy of Decision Tree classifier on validate set: {:.2f}'
     .format(clf2.score(X_validate, y_validate)))

In [ ]:
# classification reports
y_pred1 = clf1.predict(X_validate)
y_pred2 = clf2.predict(X_validate)

print('------- Model 1 -----------')
print(classification_report(y_validate, y_pred1))

print('------- Model 2 -----------')
print(classification_report(y_validate, y_pred2))

The first model (max depth 8) performed better on the training set, but the second model (max depth 4)
performed better on the validation set

# Random Forest

### 1. Fit the Random Forest classifier to your training sample and transform (i.e. make predictions on the training sample) setting the random_state accordingly and setting min_samples_leaf = 1 and max_depth = 10.

### 2. Evaluate your results using the model score, confusion matrix, and classification report.

### 3. Print and clearly label the following: Accuracy, true positive rate, false positive rate, true negative rate, false negative rate, precision, recall, f1-score, and support.

### 4. Run through steps increasing your min_samples_leaf and decreasing your max_depth.

### 5. What are the differences in the evaluation metrics? Which performs better on your in-sample data? Why?